In [ ]:
import os
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import utils as vutils
from discriminator import Discriminator
from lpips import LPIPS
from vqgan import VQGAN
from utils import load_data, weights_init
import lightning as L
from vqgan_lightning import LitVQGAN
import json
from utils import ImagePaths
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

In [ ]:
# # pull out the current working directory
# cwd = os.getcwd()

# # appened the directory with actual images
# new_path_prefix = os.path.join(cwd, "..", "processed_data", "reprocessed_resizedimages_revised.json")
# print(new_path_prefix)

# # open the json and fix it first
# json_df = pd.read_json(new_path_prefix)

# # extract negative file-paths
# negative_image_paths = json_df[json_df["time_step_label"] == 1].image_file_path.to_list()

# new_file_path = os.path.join(cwd, "..", "processed_data", "negative_image_paths.txt")
# # write out negative data
# with open(new_file_path, "w+") as f:
#     for line in negative_image_paths:
#         f.write(f"{line}\n")

In [ ]:
cwd = os.getcwd()
model_folder_path = os.path.join(cwd, "training_results", "04_03_202412_45_58")
model_file_path = os.path.join(model_folder_path, "lightning_logs", "version_0", "checkpoints", "epoch=17-step=4019112.ckpt")



negative_data_path = os.path.join(cwd, "..", "processed_data", "negative_image_paths.txt")

eval_output_path = os.path.join(model_folder_path, "negative_images")

if not os.path.exists(eval_output_path):
    os.makedirs(eval_output_path)

In [ ]:
# load negative image data
test_data = ImagePaths(negative_data_path, size=256)
negative_data_loader = DataLoader(test_data, batch_size=1, shuffle=False, num_workers=1)

In [ ]:
# load trained model
vqgan = LitVQGAN.load_from_checkpoint(model_file_path)

In [ ]:
vqgan.eval()

In [ ]:
batch_num = 0
for im in negative_data_loader:
    recon, diff = vqgan.encode_GAT_nodes(im.cuda())
    # diff = torch.abs(recons - im)

    im_np = torch.squeeze(im).permute(1,2,0).cpu().tolist()
    recons_np = torch.squeeze(recon).permute(1,2,0).cpu().tolist()
    diff_np = torch.squeeze(diff).permute(1,2,0).cpu().tolist()

    fig, axes = plt.subplots(nrows=1, ncols=3, squeeze=True)
    
    axes[0].imshow(im_np)
    axes[0].axis('off')
    axes[0].set_title("Input")
    axes[1].imshow(recons_np)
    axes[1].axis('off')
    axes[1].set_title("Recon.")
    axes[2].imshow(diff_np)
    axes[2].axis('off')
    axes[2].set_title("Diff.")
    new_path = os.path.join(eval_output_path, "neg_eval_sample_{:d}.png".format(batch_num))
    batch_num += 1
    fig.savefig(new_path)
    plt.close()

